## Project Report
This project is a solution to the Unity's reacher environment (second version with 20 agents). The environment was solved by using Deep Deterministic Policy Gradients (DDPG) algorithm after 103 episodes.

**Note**: It's best to view this report locally with Jupyter notebook or Jupyter lab. Github's Markdown display cannot display this file correctly.

### Environment
![alt text](https://video.udacity-data.com/topher/2018/June/5b1ea778_reacher/reacher.gif "Reacher")

In this environment, a double-jointed arm can move to target locations. A reward of +0.1 is provided for each step that the agent's hand is in the goal location. The goal of the agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of 33 variables corresponding to position, rotation, velocity, and angular velocities of the arm. Each action is a vector with four numbers, corresponding to torque applicable to two joints. Every entry in the action vector should be a number between -1 and 1.

### Experience replay buffer
An experience replay buffer with size of `1e6` is used to store the experience tuple. Each tuple contains the input state, the chosen action, the corresponding reward from the environment and the next state. For each training step, 256 tuples will be randomly sampled from the buffer to train the actor and critic network.

### Deep Deterministic Policy Gradients (DDPG)
Deep Deterministic Policy Gradients (DDPG) is an algorithm used for solving environments where the action values are continous. The algorithm combines the actor-critic approach with Deep Q network method in order to solve the environment. With the actor-critic method, the agent uses 2 neural network models called actor and critic. The actor $\mu(s|\theta^\mu) $ takes the input state $ s $ and outputs the action value $ a $ while the critic  $ Q(s|\theta^Q) $  acts as a non linear state value approximator which takes the input state $ s $ with the action $ a $ and output the value of action $ a $ in state $ s $. With the DQN approach, both the actor and critic networks are initialized as 2 seperate versions, local   $\mu(s|\theta^\mu) $, $ Q(s|\theta^Q) $ and target $\mu'(s|\theta^{\mu'}) $,  $ Q'(s|\theta^{Q'}) $ with identical parameters. Once the replay buffer collects enough experiences and training start, the local networks are trained by target values generated by the target networks to reduce divergence. The weights of the target networks &theta;' are then updated to slowly track their local versions using the soft update method : $ \theta' \leftarrow \tau\theta + (1-\tau)\theta' $ with $ \tau =1e-3 $.  

### Actor
The actor function $\mu(s|\theta^\mu) $ specifies the current policy by directly map the input state vector $ s $ to the output action $ a $. To approximate this function, we use a neural network with 3 layers: the input layer, 1 hidden layer with 256 nodes and the output layer. The first 2 layers use Relu activation function while the output use tanh to squash the action values between 1 and -1. Batch normalization is also applied to the hidden layer. 


### Critic
The critic function $ Q(s|\theta^Q) $  approximates the expected return when action $ s $  is taken at state $ a $. Again, we use a neural network with 4 layers: the input layer, 2 hidden layers with sizes of 512 and 384 respectively and the output layer. Leaky ReLU is used as activation function for every layer except the output. The output layer does not have an activation function and output the value of action $ a $ in state $ s $ as a real number.

**Note**: Both the actor and critic network weights needs to be initialized manually using values drawn from the random uniform distribution otherwise they will not be able to learn effectively. .From my experience, if the weights are initialized using Pytorch's default setting, the agent will not be able to reach score larger than 4.

### Exploration noise for continous action value
Following the DDPG paper, we constructed an exploration policy &micro;' by adding noise sampled from a noise process $ N $ to our actor policy $ a_t = \mu(s_t|\theta^{\mu}) + N(t) $ . $ N $ is sampled from a Ornstein-Uhlenbeck process to generate correlated noise with $ \mu=0, \theta=0.15 $ and $ \sigma=0.4 $. $ \sigma $ is then gradually reduced to 0.2 following a decaying period of 1000 steps to reduce exploration and increase exploitation as the policy get better.

### Training algorithm
>Initialize actor local network $\mu(s|\theta^\mu) $ and critic local network  $ Q(s|\theta^Q) $ with weights following the above method.  
Initialize actor target network ${\mu'}(s|\theta^{\mu'}) $ and critic target network $ {Q'}(s|\theta^{Q'}) $  as a copy of their local counterparts.
Initialize replay buffer $R$.\
>__for__ episode = 1, 2000 __do__:
>>Initialize (reset) the noise process $ N $.\
Receive initial state observation $ s_t$.\
__for__ t=1, 1000 __do__:    
>>>Select action $ a_t = \mu(s_t|\theta^\mu) + N(t) $ using the local actor $ \mu(s_t|\theta^\mu) $.\
Execute action $ a_t $, observe reward $ r_t $ and next state $ a_{t+1} $\
Store the transition $ (s_t, a_t, r_t, s_{t+1}) $ in replay buffer $ R $.\
Sample a batch of transitions $ (s_i, a_i, r_i, s_{i+1}) $ with batch size 512 from $ R $.\
Set $ Q_i = r_i + \gamma{Q'}(s_{i+1}, {\mu'}(s_{i+1}|\theta^{\mu'})) $.\
Update local critic network $ Q(s,a | \theta^Q) $ by minimizing the loss $ L = \frac{1}{N}\sum_{i}{(Q_i - {Q(s_{i}, a_{i} | \theta^Q)})^{2}} $\
Update the critic network using sampled policy gradient: $ \nabla_{\theta^\mu} {J} \approx \frac{1}{N}\sum_{i}\nabla_a {Q(s,a|{\theta^Q})|}_{s=s_i,a=\mu(s_i)}\nabla_{\theta^\mu} {\mu({s}|\theta^{\mu})|}_{s_i} $. \
Update the target network:\
$ \theta^{Q'} \leftarrow \tau\theta^Q + (1-\tau)\theta^{Q'} $\
$ \theta^{\mu'} \leftarrow \tau\theta^\mu + (1-\tau)\theta^{\mu'} $

### Hyper parameters
```python
ACTOR_LEARNING_RATE = 1e-3
CRITIC_LEARNING_RATE = 1e-3
BUFFER_SIZE = int(1e6)          # replay buffer size
BATCH_SIZE = 256                # minibatch size
UPDATE_EVERY = 2                # how often to update the network
GAMMA = 0.99                    # discount factor
TAU = 1e-3                      # for soft update of target parameters
LEARN_TIMES = 1
CRITIC_GRADIENT_CLIPPING_VALUE = 1
ACTOR_GRADIENT_CLIPPING_VALUE = 0
```

## Results
The agent was able to solve the environment (average score >= 30) after 115 episode:  
![alt text](https://i.ibb.co/GvZ3QPL/agent-score.png "Average agent score")

## Future improvement
* Importance sampling for Replay buffer $ R $.
* Try to solve the task using other algorithms such as Trust Region Policy Optimization (TRPO) or Proximal Policy Optimization (PPO).